In [19]:
import pandas as pd
import datarobot as dr
import boto3
import yaml
import os
from botocore.exceptions import ClientError

In [56]:
# Get the credentials from AWS account
aws_access_key_id=""
aws_secret_access_key=""
aws_session_token=""

In [57]:
# Function which p
def copy_demo_tables_to_s3(
    aws_access_key_id = None,          # 
    aws_secret_access_key = None,      # 
    aws_session_token = None,          # 
    region_name = None,                # 
    bucket_name = None,                # Warehouse
    folder_name = None                 # schema
):
    '''description: method to copy demo table to s3
    reads from datasets.yaml 

    '''
    session = boto3.Session(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token,
        region_name=region_name
    )
    with open('datasets.yaml') as f:
        config = yaml.safe_load(f)
        for key, value in config['datasets'].items():
            print("*"*30)
            print("table:", value['table_name'])
            try:
                df = pd.read_csv(value['url'], encoding = 'utf8')
            except:
                df = pd.read_csv(value['url'], encoding = 'cp850')
            display(df.head())
            print("info for ", value['table_name'])
            print(df.info())
            print("writing", value['table_name'], "to s3 from: ", value['url'])
            file_name = value['table_name'] + '.csv'
            df.to_csv(file_name, index=False)
            s3_obj_name_model = folder_name + '/' + file_name
            try:
                s3.meta.client.upload_file(file_name, bucket_name, s3_obj_name_model)
                print("S3 Upload Complete!")
            except ClientError as e:
                print(e)

In [55]:
copy_demo_tables_to_s3(aws_access_key_id
                       , aws_secret_access_key
                       , aws_session_token
                       , region_name = 'us-east-2'
                       , bucket_name = 'private-demo-datasets'
                       , folder_name = 'ai-accelerators/test'
                      )

******************************
table: LC_PROFILE


,CustomerID,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,purpose,zip_code,addr_state
0,C900002437,4000,4000,60 months,7.29%,79.76,A,A4,Time Warner Cable,10+ years,MORTGAGE,50000.0,not verified,medical,766xx,TX
1,C900006073,16000,16000,60 months,18.25%,408.48,F,F1,Ottawa University,< 1 year,RENT,39216.0,not verified,debt_consolidation,660xx,KS
2,C900007834,8700,8700,36 months,7.88%,272.15,A,A5,Kennedy Wilson,4 years,RENT,65000.0,not verified,credit_card,916xx,CA
3,C900001691,18000,18000,60 months,11.49%,395.78,B,B4,TOWN OF PLATTEKILL,10+ years,MORTGAGE,57500.0,not verified,debt_consolidation,124xx,NY
4,C900002594,16000,16000,36 months,11.83%,530.15,B,B3,Belmont Correctional,10+ years,MORTGAGE,50004.0,VERIFIED - income,debt_consolidation,439xx,OH


info for  LC_PROFILE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           10000 non-null  object 
 1   loan_amnt            10000 non-null  int64  
 2   funded_amnt          10000 non-null  int64  
 3   term                 10000 non-null  object 
 4   int_rate             10000 non-null  object 
 5   installment          10000 non-null  float64
 6   grade                10000 non-null  object 
 7   sub_grade            10000 non-null  object 
 8   emp_title            9408 non-null   object 
 9   emp_length           9741 non-null   object 
 10  home_ownership       10000 non-null  object 
 11  annual_inc           9999 non-null   float64
 12  verification_status  10000 non-null  object 
 13  purpose              10000 non-null  object 
 14  zip_code             10000 non-null  object 
 15  addr_state      

,CustomerID,AccountID,Date,Amount,Description
0,C900000001,A484993284,2016-07-21,79.42,alcohol
1,C900000001,A651639077,2016-07-31,37.87,government charges
2,C900000002,A355056969,2016-06-29,4.92,amortisation
3,C900000002,A355056969,2016-07-01,18.97,interest on purchases
4,C900000002,A355056969,2016-07-02,29.06,charity


info for  LC_TRANSACTIONS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412459 entries, 0 to 412458
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   CustomerID   412459 non-null  object 
 1   AccountID    412459 non-null  object 
 2   Date         411129 non-null  object 
 3   Amount       403750 non-null  float64
 4   Description  412441 non-null  object 
dtypes: float64(1), object(4)
memory usage: 15.7+ MB
None
writing LC_TRANSACTIONS to s3 from:  https://s3.amazonaws.com/datarobot_public_datasets/ai_accelerators/LC_transactions.csv
S3 Upload Complete!
******************************
table: LC_TRAIN


,CustomerID,BadLoan,date
0,C900000001,No,2016-08-06
1,C900000002,No,2016-07-27
2,C900000003,No,2016-07-06
3,C900000004,No,2016-08-26
4,C900000005,No,2016-08-28


info for  LC_TRAIN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9499 entries, 0 to 9498
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  9499 non-null   object
 1   BadLoan     9499 non-null   object
 2   date        9499 non-null   object
dtypes: object(3)
memory usage: 222.8+ KB
None
writing LC_TRAIN to s3 from:  https://s3.amazonaws.com/datarobot_public_datasets/ai_accelerators/LC_train.csv
S3 Upload Complete!
******************************
table: LC_TEST


,CustomerID,BadLoan,date
0,C900009501,No,2016-08-03
1,C900009502,No,2016-07-23
2,C900009503,Yes,2016-08-29
3,C900009504,Yes,2016-07-01
4,C900009505,No,2016-08-13


info for  LC_TEST
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  500 non-null    object
 1   BadLoan     500 non-null    object
 2   date        500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB
None
writing LC_TEST to s3 from:  https://s3.amazonaws.com/datarobot_public_datasets/ai_accelerators/LC_test.csv
S3 Upload Complete!
